### 타빌리 검색은 첫 1000번의 api 콜은 무료, .env에 적어둠

In [ ]:
# 타빌리 설치
%pip install tavily-python

In [ ]:
from langchain_community.tools import TavilySearchResults

tavily_search = TavilySearchResults(
    max_results=5,
)

In [ ]:
# tavily에 질문
res = tavily_search.invoke({"query": "2025 한국 경제 전망"})
for r in res:
    print(r)

In [ ]:
# 주피터 노트북에서 보기 좋게 출력
from IPython.display import JSON
JSON(res).data

In [ ]:
# 웹 페이지 전문 불러오기
from tavily import TavilyClient

client = TavilyClient()

content = client.search(
    "2025년 한국 경제 전망",
    search_depth="advanced", # basic 보다 정확하고 깊이 있는 검사
    include_raw_content=True,
)

JSON(content).data

### 인터넷에서 자료 조사 후 기사 쓰는 기자 만들기

In [ ]:
# 질문에 대한 자료 검색하기
from langchain.adapters.openai import convert_openai_messages
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

# 질문에 대한 자료를 TavilyClient로 가져오기
query = "현대자동차 미국 시장 2025년 전망"

content = client.search(
    query,
    include_raw_content=True,
    search_depth="advanced", # basic 보다 정확하고 깊이 있는 검사
)["results"]

In [ ]:
# 프롬프트 설정
prompt = [{
    "role": "system",
    "content": (
        "당신은 신문 기사를 쓰는 기자 AI입니다. \n"
        "당신은 주어진 정보를 바탕으로 객관적이고 체계적으로 작성된 기사를 써야 합니다. \n"
    )
}, {
    "role": "user",
    "content": (
        f'정보: """{content}"""\n\n'
        f'위의 정보를 사용하여, 다음 질문에 대해 자세한 보고서를 한국어로 작성하세요.: "{query}" \n'
        '-신문 기사 형식을 사용하되, MLA 표준을 준수하는 markdown 문법을 사용해주세요.'
        '-활용한 자료는 출처를 명시하세요.'
    )
}]

In [ ]:
# 오픈ai를 랭체인으로 실행하고 출력하기
report = llm.invoke(prompt).content
print(report)

# 셀 실행 결과를 copy cell output으로 저장하고 .md 파일에 저장, ai 뉴스 생성 사이트 제외가 잘 안되네.